In [21]:
// Sempre que precisamos de fazer Console.ReadLine num jupiter notebook precisamos de correr esta célula uma vez.
// Não apagues pois podemos vir a precisar dela.
static class Console {
    public static void WriteLine(object o) => System.Console.WriteLine(o);
    public static void WriteLine(bool b) => System.Console.WriteLine(b);
    public static void WriteLine(string text) => System.Console.WriteLine(text);
    public static void WriteLine(int n) => System.Console.WriteLine(n);
    public static string ReadLine() => GetInputAsync("").GetAwaiter().GetResult();
}

# Sistema de Gestão de Residência Universitária
**Projeto POO 2025 – Parte Inicial**

**1. Descrição Geral**
O objetivo é desenvolver uma aplicação para gerir o alojamento de estudantes numa residência.
O sistema deve controlar a ocupação dos diferentes tipos de quartos e manter o registo
atualizado dos residentes e da sua condição financeira/social.

**2. Hierarquia de Classes e Requisitos**
Alojamento (Quartos)
Deve implementar uma estrutura de classes para representar os quartos, utilizando o conceito
de herança:
-  **Alojamento:** Contém atributos comuns como número do quarto, área base e preço base,
mas não pode ser instanciado.
-  **Quarto Partilhado:** Define o número de camas/lugares (entre 2 e 4). O preço base é
dividido pelos lugares. Há um desconto de 10% por cada lugar adicional (aplicável apenas
a 3 (10%) e 4 lugares (20%)).
- **Quarto Individual:** Quarto para apenas um ocupante. Tem um custo adicional de 10%
face ao preço base do quarto.
- **Quarto Individual Premium:** Herda de Quarto Individual, mas permite guardar uma área
superior. Além do custo de quarto individual, paga mais 4% do valor do quarto, por cada
m2 extra de área.
Estudantes e Apoio Social
A gestão de utilizadores deve distinguir os estudantes com base nas suas condições económicas:
- **Estudante:** Nome, NIF, Instituição de Ensino, Rendimento Mensal do Agregado e
Dimensão do Agregado Familiar.
- **Estudante com Apoio Social:** Inclui um Escalão de Apoio e a taxa de desconto na
mensalidade.

**Regra de Negócio (Cálculo do Escalão):**

O escalão é atribuído automaticamente com base no Rendimento per Capita (Rpc):
Rpc = Rendimento do Agregado / Dimensão do Agregado
- Escalão A: Rpc < 300€ - Desconto máximo de 66%
- Escalão B: 300€ <= Rpc < 450€ - Desconto Máximo de 50%
- Escalão C: 450€ <= Rpc <= 600€ - Desconto Máximo de 35%
- Não Carenciado: Rpc > 600€ (Deve ser tratado como Estudante comum).

A taxa de desconto depende do alojamento atribuído. Quando afeto a um alojamento, esta
taxa deve ser atualizada, sendo que o estudante apenas pode pagar 50% do seu Rpc. A
diferença entre o valor do alojamento e o valor pago, deve ser usada para calcular a taxa de
desconto.

**3. Interfaces**
Devem ser implementadas as seguintes Interfaces:

***ICalculavel (implementada pelos alojamentos):***
- double Mensalidade() – Devolve o valor da mensalidade por ocupante;
- double Mensalidade(Estudante ocupante) – Devolve o valor da mensalidade para o ocupante
(atribui eventual desconto na mensalidade para estudantes com Apoio Social);
- double Ocupacao() – devolve a taxa de ocupação do alojamento (entre 0 e 1).

***IRendimento (implementada pelos estudantes):***
- double RendimentoPerCapita() – devolve o rendimento do estudante, considerando o valor e
dimensão (ver fórmula acima).

**4. Aplicação**
 – Deve ser criada uma “Residência” com 20 alojamentos
Cada grupo define a proporção quantidade existente de cada tipologia de alojamentos, mas
todos devem existir.
Cada grupo deve ainda definir como associar os estudantes aos alojamentos. Mas deve ser
possível perceber onde está cada estudante alojado, e quem está alojado em cada quarto/lugar.
A aplicação deve criar os alojamentos e depois ter uma interface que permita registar novos
estudantes (pedindo os dados ao utilizador), qualquer que seja o tipo, e permitir a escolha
atribuição de quarto, de acordo com a tipologia e disponibilidade.
Na atribuição de alojamentos a estudantes carenciados, a aplicação deve impedir que esta
aconteça se o valor absoluto do “desconto” for superior ao Rendimento per Capita do estudante.

**5. Exceções**


5. As Exceções
Devem ser tratadas as situações de eventuais erros, capturando e lançando exceções.
------------------------------------------------

**Funcionalidades Esperadas**

1. Registo e Listagem: Adicionar novos quartos e estudantes ao sistema.
2. Atribuição de Alojamento: Validar se um quarto partilhado ainda tem vagas antes de
alocar um estudante. E se os individuais estão livres.
3. Cálculo de Mensalidade: Definido na Interface ICalculavel.
4. Estatísticas: Listar total de rendimento mensal da residência e percentagem de
ocupação.

In [1]:
 public abstract class Alojamento(){
    public int NumeroQuarto {get;set;}
    public double AreaBase {get;set;}
    private double PrecoBase {get;set;} 
       
 }
 


 public class QuartoPartilhado : Alojamento {
 }
 
 public class QuartoIndividual : Alojamento {
 }
 
  public class QuartoPremium : QuartoIndividual{
 }
 


 interface ICalculavel {
    double Mensalidade();
    double Ocupacap();
    
}

 interface IRendimento {
    double RendimendoPerCapita();
    
}